In this notebook I process the .csv transaction data dump from commsec to do some cash flow analysis and stuff for my mum.

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from yahoo_finance import Share

I went to Trading>Confirmations and downloaded a .csv file.

In [4]:
confirmations = pd.read_csv('ConfirmationDetails.csv').iloc[:, :11]
confirmations.head(2)

,Confirmation Number,Order Number,Trade Date,Buy/ Sell,Security,Units,Average Price ($),Brokerage (inc GST.),Net Proceeds ($),Settlement Date,Confirmation Status
0,75757719,N89211655,5/01/2017,S,SOL,1298,15.40,29.95,19959.25,9/01/2017,Confirmed
1,75740722,N89059094,4/01/2017,S,KDR,15800,0.63,19.95,9934.05,6/01/2017,Confirmed


In [5]:
current_prices = {}

for ticker in confirmations.Security.unique():
    current_prices[ticker] = Share(ticker+'.AX').get_price()

current_prices = pd.DataFrame.from_dict(current_prices, orient='index')
current_prices.columns = ['price']
current_prices.sort_values('price', ascending=False, inplace=True)

I went to Portfolio>Accounts>Transactions and downloaded a .csv of all the transactions there.

Had to make sure that my date range started at June 1, to get all the info.

In [6]:
transactions = pd.read_csv("Transactions.csv").iloc[:, :7]
transactions.fillna("", inplace=True)
transactions['Balance ($)'] = transactions['Balance ($)']/1000
transactions.rename(columns={'Balance ($)': "Balance ($000)"}, inplace=True)
transactions.head(2)

,Date,Reference,Type,Detail,Debit ($),Credit ($),Balance ($000)
0,4/01/2017,C75740722,Contract,S 15800 KDR @ 0.630000,,9934.05,-41.59523
1,4/01/2017,C75739773,Contract,S 16129 KDR @ 0.620000,,9980.03,-31.66118


In [7]:
divs = pd.read_csv('CSVData.csv', header=None)
divs['Security'] = divs.iloc[:,2].apply(lambda x: x.split()[3])
divs.rename(columns={1: 'Proceeds'}, inplace=True)
divs.head(2)

,0,Proceeds,2,3,Security
0,21/12/2016,130.66,Direct Credit 250556 WBC DIVIDEND 001196002298,47345.79,WBC
1,13/12/2016,209.88,Direct Credit 002473 NAB FIN DIVIDEND DV172/00...,6206.31,NAB


In [8]:
big_dict = {}

for ticker in confirmations.Security.unique():
    revenue = confirmations['Net Proceeds ($)'][
        confirmations['Security'] == ticker][
        confirmations['Buy/ Sell']=='S'].sum()

    cost = confirmations['Net Proceeds ($)'][
            confirmations['Security'] == ticker][
            confirmations['Buy/ Sell']=='B'].sum()
    
    tran_num = confirmations[confirmations.Security == ticker].shape[0]
    
    num_bought = confirmations['Units'][(confirmations['Security'] == ticker) & (confirmations['Buy/ Sell'] == 'B')].sum()
    num_sold = confirmations['Units'][(confirmations['Security'] == ticker) & (confirmations['Buy/ Sell'] == 'S')].sum()
    num_holding = num_bought-num_sold
    
    curr_price = float(current_prices.ix[ticker])
    asset_value = curr_price * num_holding
    if divs.Proceeds[divs.Security == ticker].shape[0] == 0:
        dividend = 0
    else: dividend = round(divs.Proceeds[divs.Security == ticker].values.tolist()[0], 2)
            
    prof=round(revenue+int(dividend)-cost, 2)
    net_position = round(prof + asset_value, 2) # is this right? am i double counting something?
    
    big_dict[ticker] = [round(revenue, 2), round(cost, 2), tran_num, num_holding,
                        asset_value, dividend, prof, net_position, curr_price]

In [9]:
all_info = pd.DataFrame.from_dict(big_dict, orient='index')
all_info.columns = ['Revenue', 'Cost', 'Num Transactions', 'Num Holding', 'Asset Value', 'Dividends', 'Profit', 'Net Position', 'Current Price']
all_info.sort_values('Net Position', ascending=True)

,Revenue,Cost,Num Transactions,Num Holding,Asset Value,Dividends,Profit,Net Position,Current Price
OVN,67591.57,88225.06,13,8530,5672.450,0.00,-20633.49,-14961.04,0.665
MZN,15612.05,23492.95,2,0,0.000,0.00,-7880.90,-7880.90,0.022
SYR,136578.50,143703.12,22,0,0.000,0.00,-7124.62,-7124.62,3.340
BLK,223909.07,240712.13,31,19603,10291.575,0.00,-16803.06,-6511.49,0.525
ACX,99161.92,105255.21,11,0,0.000,0.00,-6093.29,-6093.29,5.450
MYX,21255.16,26501.03,3,0,0.000,0.00,-5245.87,-5245.87,1.400
SOL,177340.39,181891.98,28,0,0.000,962.55,-3589.59,-3589.59,15.280
SAR,0.00,10018.26,1,7407,7851.420,0.00,-10018.26,-2166.84,1.060
RFG,109747.00,111901.90,13,0,0.000,0.00,-2154.90,-2154.90,7.030
EVN,20988.05,30644.90,3,3800,8018.000,0.00,-9656.85,-1638.85,2.110


In [10]:
all_info['Net Position'].sum()

-19278.91

In [11]:
#     print("""For {ticker}: 
#     ${spent} in total has been spent, ${gained} has been gained from sales and ${divi} from dividends.  
#     {tran_num} transactions were undertaken for {ticker2}.
#     We currently hold {num_holding} units, valued at ${asset_value} total.
#     This represents ${prof} net profit.
#     Your net position on this stock is ${net}.
#     """.format(ticker=ticker,
#                spent=round(cost, 2),
#                gained=round(revenue, 2),
#                divi=int(dividend),
#                prof=prof,
#                tran_num=tran_num,
#                ticker2=ticker,
#                num_holding=num_holding,
#                asset_value=asset_value,
#                net=net_position))